In [1]:
"""
Here we try to train heavy pretrained xception65 model for rupture detection problem


Problems: 1)Problem: It wasn't problematik to solve problem with load nets module
            Solution: You have to go in research/slim directory and replace all entries: from nets... to fro slim.nets...
          2)Problem: It not compofortable to operate wth bash commmand and environment variables inside Ipython notebook
            Solution: Anywhere where it is possible the bash commands are changed to python3 wrapper (mkdit to os.makedirs and so on...)

"""


"\nHere we try to train heavy pretrained xception65 model for rupture detection problem\n\n\nProblems: 1)Problem: It wasn't problematik to solve problem with load nets module\n            Solution: You have to go in research/slim directory and replace all entries: from nets... to fro slim.nets...\n          2)Problem: It not compofortable to operate wth bash commmand and environment variables inside Ipython notebook\n            Solution: Anywhere where it is possible the bash commands are changed to python3 wrapper (mkdit to os.makedirs and so on...)\n\n"

In [2]:
"""
!pip install pydrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""

'\n!pip install pydrive\n\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n'

In [3]:

#GPU test

import tensorflow as tf
from tensorflow.python.client import device_lib

#tf.test.gpu_device_name()
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9396935942596443399, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7326689066636995939
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 511702918441324594
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11276946637
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14899409078373536769
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [4]:
# Change directory to project folder on google drive instance

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd "/content/drive/My Drive/research/deeplab"

import os
os.environ['CURRENT_DIR']= "/content/drive/My Drive/research"
#%env WORK_DIR="$CURRENT_DIR/deeplab"
os.environ['WORK_DIR'] = os.environ['CURRENT_DIR'] + "/deeplab"
os.environ['DATASET_DIR'] = os.environ['WORK_DIR'] + "/datasets"
#%env DATASET_DIR="$WORK_DIR/datasets"



/content/drive/My Drive/research/deeplab


In [0]:
os.environ['PYTHONPATH'] = "/content/drive"

In [0]:
#Note: you have to add custom libraries to python path FROM research directory(see CURRENT_DIR)


os.chdir(os.environ['WORK_DIR']) 
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/research"
#%cd $CURRENT_DIR"/slim"
os.environ['PYTHONPATH'] += ":/content/drive/My Drive/research/slim"

#os.environ['PYTHONPATH'] += ":/content/drive/My\ Drive/research/deeplab"
#os.environ['PYTHONPATH'] += ":/content/drive/My Drive/research/slim/nets"


In [8]:
!ls

common.py	    input_preprocess.py        vis.py
common_test.py	    local_test_mobilenetv2.sh  WZ_eval.py
core		    local_test.sh	       WZ_export_model.py
datasets	    model.py		       WZ_local_test.sh
deeplab_demo.ipynb  model_test.py	       WZ_local_test_xception65.sh
eval.py		    __pycache__		       WZ_model_test.py
export_model.py     README.md		       WZ_vis.py
g3doc		    train.py
__init__.py	    utils


##Create Environment and load pretrained model from model zoo

In [10]:




# Set up the working directories.
os.environ['RUPTURE_FOLDER'] =os.environ['DATASET_DIR']    + '/WZL_rupture_seg'
os.environ['EXP_FOLDER']     =os.environ['RUPTURE_FOLDER'] + '/exp/train_on_trainval_set'

os.environ['INIT_FOLDER']    =os.environ['RUPTURE_FOLDER'] + '/init_models'
os.environ['TRAIN_LOGDIR']   =os.environ['EXP_FOLDER']     + '/train'
os.environ['EVAL_LOGDIR']    =os.environ['EXP_FOLDER']     + '/eval'
os.environ['VIS_LOGDIR']     =os.environ['EXP_FOLDER']     + '/vis'
os.environ['EXPORT_DIR']     =os.environ['EXP_FOLDER']     + '/export'

try:
  os.makedirs(os.environ['INIT_FOLDER'] ,  mode=777,  exist_ok=True)
  os.makedirs(os.environ['TRAIN_LOGDIR'],  mode=777,  exist_ok=True)
  os.makedirs(os.environ['EVAL_LOGDIR'] ,  mode=777,  exist_ok=True)
  os.makedirs(os.environ['VIS_LOGDIR']  ,  mode=777,  exist_ok=True)
  os.makedirs(os.environ['EXPORT_DIR']  ,  mode=777,  exist_ok=True)
except:
  raise ValueError('Cant create directories')



# Copy locally the trained  checkpoint as the initial checkpoint.
os.environ['TF_INIT_ROOT'] = "http://download.tensorflow.org/models"
os.environ['TF_INIT_CKPT'] = "deeplabv3_pascal_train_aug_2018_01_04.tar.gz"                #Heavy pretrained model-Xception65

#TF_INIT_CKPT="deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz"          #Light pretrained model
os.chdir(os.environ['INIT_FOLDER'])
!wget -nd -c "$TF_INIT_ROOT/$TF_INIT_CKPT"
!tar -xf "$TF_INIT_CKPT"

#%cd $CURRENT_DIR

os.chdir(os.environ['WORK_DIR'])
os.environ['RUPTURE_DATASET']=os.environ['RUPTURE_FOLDER']+ "/tfrecord"



--2019-03-03 12:50:08--  http://download.tensorflow.org/models/deeplabv3_pascal_train_aug_2018_01_04.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.126.128, 2607:f8b0:4001:c1d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.126.128|:80... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
#os.chdir(os.environ['CURRENT_DIR'] + '/slim')

#!python -c "from nets import cifarnet; mynet = cifarnet.cifarnet"

In [0]:
#!python -c "import tensorflow.contrib.slim as slim; eval = slim.evaluation.evaluate_once"


In [0]:
os.chdir(os.environ['WORK_DIR'])
os.environ['NUM_ITERATIONS'] = '40'

##Call train.py file and perform train procedure

In [16]:


!echo "=========================== Begin the training ============================"

# Train 10 iterations.
#NUM_ITERATIONS=10

!python train.py \
  --logtostderr \
  --train_split="train" \
  --dataset="rupture" \
  --model_variant="xception_65" \
  --output_stride=16 \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --fine_tune_batch_norm=False \
  --train_crop_size=513 \
  --train_crop_size=513 \
  --train_batch_size=4 \
  --decoder_output_stride=4 \
  --training_number_of_steps="${NUM_ITERATIONS}" \
  --tf_initial_checkpoint="${INIT_FOLDER}/deeplabv3_pascal_train_aug/model.ckpt" \
  --train_logdir="${TRAIN_LOGDIR}" \
  --dataset_dir="${RUPTURE_DATASET}" \
  --initialize_last_layer=False \
  --last_layers_contain_logits_only=False


 

#--model_variant="xception_65" \

#--atrous_rates=6 \
#  --atrous_rates=12 \     only
#  --atrous_rates=18 \     for
#                          xception65

#--fine_tune_batch_norm=False \
#--last_layers_contain_logits_only = False
#
#--tf_initial_checkpoint="${INIT_FOLDER}/deeplabv3_pascal_train_aug/model.ckpt" \               #Heavy pretrained model

!echo "=========================== End of  the training ============================"


=========================== Begin the training ============================

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Training on train set
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(t

In [0]:
!echo "=========================== Begin of the evaluation ============================"


!python WZ_eval.py \
  --logtostderr \
  --eval_split="val" \
  --model_variant="xception_65" \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --output_stride=16 \
  --decoder_output_stride=4 \
  --eval_crop_size=2454 \
  --eval_crop_size=2454 \
  --fine_tune_batch_norm=False \
  --dataset="rupture" \
  --checkpoint_dir="/content/drive/My Drive/research/deeplab/datasets/WZL_rupture_seg/exp/train_on_trainval_set/train" \
  --eval_logdir="/content/drive/My Drive/research/deeplab/datasets/WZL_rupture_seg/exp/train_on_trainval_set/eval" \
  --dataset_dir="/content/drive/My Drive/research/deeplab/datasets/WZL_rupture_seg/tfrecord" \
  --max_number_of_evaluations=1


#--model_variant="xception_65" \
#--atrous_rates=6 \
#--atrous_rates=12 \
#--atrous_rates=18 \
# --decoder_output_stride=4 \
#--eval_batch_size=1 \
!echo "=========================== End of the evaluation ============================"



=========================== Begin of the evaluation ============================

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Evaluating on val set
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tens

In [0]:
#Error

!echo "============================Begin of the Visualization=========================="

# Visualize the results.
!python WZ_vis.py \
  --logtostderr \
  --vis_split="val" \
  --dataset="rupture" \
  --model_variant="xception_65" \
  --output_stride=16 \
  --vis_crop_size=513 \
  --vis_crop_size=513 \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --fine_tune_batch_norm=False \
  --decoder_output_stride=4 \
  --checkpoint_dir="${TRAIN_LOGDIR}" \
  --vis_logdir="${VIS_LOGDIR}" \
  --dataset_dir="${RUPTURE_DATASET}" \
  --max_number_of_iterations=1


#--model_variant="xception_65" \
#--atrous_rates=6 \
#--atrous_rates=12 \
#--atrous_rates=18 \
# --decoder_output_stride=4 \
!echo "=========================== End of the Visualization =================-========================"





In [17]:

!echo "=========================== Begin of the export of retrained model ============================"

# Export the trained checkpoint.
os.environ['CKPT_PATH']=os.environ['TRAIN_LOGDIR'] + '/model.ckpt-' + os.environ['NUM_ITERATIONS']
os.environ['EXPORT_PATH']=os.environ['EXPORT_DIR'] + '/frozen_inference_graph.pb'

!python WZ_export_model.py \
  --logtostderr \
  --dataset="rupture" \
  --checkpoint_path="${CKPT_PATH}" \
  --export_path="${EXPORT_PATH}" \
  --model_variant="xception_65" \
  --output_stride=16 \
  --num_classes=2 \
  --crop_size=2454 \
  --crop_size=2454 \
  --atrous_rates=6 \
  --atrous_rates=12 \
  --atrous_rates=18 \
  --decoder_output_stride=4 \
  --inference_scales=1.0











#--model_variant="xception_65" \
#--atrous_rates=6 \
#--atrous_rates=12 \
#--atrous_rates=18 \
# --decoder_output_stride=4 \
!echo "=========================== End of the exporting of retrained model ============================"

=========================== Begin of the export of retrained model ============================
INFO:tensorflow:Prepare to export model to: /content/drive/My Drive/research/deeplab/datasets/WZL_rupture_seg/exp/train_on_trainval_set/export/frozen_inference_graph.pb
INFO:tensorflow:Exported model performs single-scale inference.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2019-03-03 12:57:21.122478: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-03 12:57:21.122742: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1fe9180 executing computations on platform Host. Devices:
2019-03-03 12:57:21.122779: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-03 12:57:21.190878: I tensorflow/stream_executor/cuda/cuda_